In [ ]:
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib
!pip install google-auth-oauthlib==0.4.6
!pip install crewai_tools
!pip install python-docx

!pip install PyPDF2

In [ ]:
#mount to drive
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Email_Agent/

In [ ]:
import pickle
import google.auth
from google.auth.transport.requests import Request
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
import base64
from crewai.tools import BaseTool
from googleapiclient.discovery import Resource
from pydantic import BaseModel, Field
import os
import tools
from tools import *


os.environ["OPENAI_API_KEY"]= 'your key'


global service
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly',
          'https://www.googleapis.com/auth/gmail.send',
          'https://www.googleapis.com/auth/gmail.modify']

creds = None

#Lưu token đăng nhập vào file token.pkl
if os.path.exists('token.pkl'):
    with open('token.pkl', 'rb') as token:
        creds = pickle.load(token)

# Nếu chưa có hoặc hết hạn thì yêu cầu đăng nhập
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
       # Chạy flow đăng nhập thủ công: bạn sẽ nhận 1 link và nhập mã xác nhận
          flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
          # Sử dụng run_console thay vì run_local_server cho môi trường không có trình duyệt
          creds = flow.run_console()

with open('token.pkl', 'wb') as token:
  pickle.dump(creds, token)

service = build('gmail', 'v1', credentials=creds)



In [ ]:
tools.set_service(service)
from crewai import Agent, Crew, Task, LLM



email_screener_agent = Agent(
    role="Email Classification, Initial Screening Specialist and Respone to Candidate",
    goal=(
        '''classify job application emails from other types of emails and perform initial screening
    based on professionalism, courtesy in communication, and completeness of application materials
    (presence of attached CV). Check if the company is currently hiring for the position and if this candidate has applied for it. Make decisions (restore, pass, need additional submission, reject).'''
    ),
    backstory=(
        """You are an HR administrative specialist with 6 years of experience in handling emails and job applications.
    you have the ability to identify job application emails from other types of emails,
    assess professionalism in email writing, check completeness of applications and can quick check job opening

    You are NOT responsible for evaluating candidates' professional skills or qualifications -
    your role stops at ensuring emails are properly formatted, courteous, and contain necessary documentation and contain in job list.
    You can't write an email, you just decide whether to write an email for a particular situation or not

    You always maintain a fair, objective attitude and base decisions solely on formal criteria. Especially do not deal with rude, unprofessional emails
    For borderline cases (like candidates have polite and profestional email content but they fogot attach CV - just accident),
    you tend to give candidates a second chance if they demonstrate sincerity."""
    ),
    #thêm vào đây nếu email đầy đủ hợp lý nhưng không có trong job list thì lưu vào 1 thư mục và viết mail thông báo sẽ xem xét tuyển lại sau
    tools=[
        CheckAttachmentTool(),
        ReadEmailContentTool(),
        ReadFileTool(),
        ReadEmailTitleTool(),
    ],
    llm = 'gpt-4.1',
    verbose=False,
    max_iter = 10
)
llm = LLM(
    model="openai/gpt-4.1", # call model by provider/model_name
    temperature=0.3,# set low temp to ensure same context for multi CV

)

cv_check_biz_agent = Agent(
    role="professional talent acquisition in business-finance-marketing field",
    goal=(
        '''explore jobs folder environment, read JD + Candidate CV to score them who in the field'''
    ),
    backstory=(
        """
You are a recruitment professional with experience in specialized industries.
Always be clear about the candidate level you are hiring for.
You can deeply analyze the JD and CV using your professional expertise.
Don't be misled by fancy wording or a dense list of shallow skills. — focus on what the candidate has actually achieved, the deep, the results delivered, the creativity, and the difficulty level relative to their seniority.
Listing skills without providing evidence or relevant experience will not be highly valued.
Be thoughtful when evaluating cross-domain profiles: some outstanding achievements may justify the switch, while others may not.
Quality matters more than quantity. A strong candidate shows real impact, not just long lists of job titles or skills.
Your goal is to identify those who can deliver value in practice, who have potential, not just look good on paper."""
    ),
    llm = llm,
    tools=[
        ReadFileTool(),
        ExploreDirectoryTool(),
        SendEmailsTool(),
        MarkReadedTool(),
        SaveFileTool()
    ],
    verbose=False,
    max_iter=15
)




In [ ]:
from crewai import Agent, Crew, Task, Process
tools.set_service(service)


#use email api to get these email ID
email_id = "1985407376d565df" # co cv 19780f9fd2cc139c, khong cv: 19780f9124d2105d, so sai: 1975cd23f0abe5a0, sale 1985407376d565df
input_task = Task(
    description=f"""
    You need to perform the preliminary filtering process of application emails according to the following steps with email_id: {email_id}

    STEP 1: READ AND CLASSIFY EMAIL
    - Read the title and content of the email from the given email_id (call tools)
    - Determine whether this is an application email or not, if yes, go to the next step, if not, end and notify that it is not an application email.

    STEP 2: EVALUATE THE QUALITY OF EMAIL (applicable if it is an application email)
    - Title is clear, specific, and mentions the position applied for
    - Email content is polite and professional, including: greeting, self-introduction and application position, reason for application, briefly state skills (optional) and wishes + thanks.
    - Check and save the attached file if any (call tool), remember the path to file then read the file (call tool) to see if it is a CV, if it is suitable for open positions in list jobs (note: only read briefly, do not analyze the expertise)
    - understand what exact position candidate apply for.

    STEP 3: GET INFORMATION ABOUT JOB OPENING
    - read the file '/content/drive/MyDrive/Email_Agent/joblist.txt' (call tool) to get and understand list of jobs available (job title + level)

    STEP 4: PROCESSING CASE-BY-CASE
    - Is the position this candidate is applying for currently open, if not: restore candidate (carefully consider)
    - If the title, content of the email are sketchy and unprofessional no quality- egardless of whether or not attach a CV: Notify rejection + need email reject
    - If the title and content are polite, professional and good-natured but the CV is not attached (maybe an accident): Notify additional submission + need email submission.
    - If the attached content is not a CV or the CV content does not match the position applied for: Notify rejection + need email reject.
    - If the title, content, and CV are guaranteed: candidate pass + no need email for this round
    **notice: the first prior is the position candidate apply is open or not and level of position, the second prior is the professional of email - not CV so if it there is no polite, no professional: reject

    You are provided with tools including reading the title, reading the email content, checking and saving attachments and reading files. Use them flexibly and proficiently
    """,
    agent=email_screener_agent,
    expected_output="the candidate [restore] / [pass or reject + email reject or need additional submission + email submission] in [screen round] with [position]"
)


scoring_task = Task(
    description=f"""
The candidate has passed the email scan round. Now rate the professional skills of this candidate's CV.
First, you explore the folders of jobs that are recruiting [each folder represents 1 job, in each job there will be JD and scoring_criteria]
Pay attention to the folder name
Evaluate the candidate's CV according to the following steps:
(1) read the JD file of the job that candidate is interested in, analyze the must-haves, nice to haves, and visualize the jobs the candidate will do
(2) Read candidate's CV, analyze and preliminarily score projects or experiences (depending on level) to see if they are suitable, are potential for JD or not. This score is on a 30-point scale (criteria 0)
(3) Next, go deeper into the skills. Read the scoring_criteria.txt file (n more criteria) and score the CV according to the criteria.

#Note 1: Don't let the fancy words in the CV fool you, focus on what the candidate actually does, the results, creative as well as the depth of the projects, skill or work these criteria are measured on a scale of 10 step 0.5.
Listing skills without providing evidence or relevant experience will not be highly valued.
Evaluate according to the level of the candidate, intern will be different from junior, middle or senior.
#Note 2: Must follow the main expertise of the JD,
Example:
  JD about electronic equipment trading, the sales skills score of the candidate working in the field of laptop sales will be higher than the candidate working in the field of real estate sales, even though the same number of years of experience.
  JD about NLP, the Python programming skills score of the candidate with a project on emotion analysis will be higher than the candidate with a project on object recognition, even though the 2 projects have the same difficulty and the same programming language
then give the candidate's name, score of each criterion (including criteria 0) and email
    """,
    agent = cv_check_biz_agent,
    context = [input_task],
    # nên để các tiêu chí chấm điểm trong JD
    expected_output="Expect candidate name, score of each criteria + explanation and email"
)# xem xét phương thức đánh giá



selecting_task = Task(
    description=f"""
    The candidate's CV has been scored with the criteria and explained why.
    Calculate the total score of the CV and decide whether to let this candidate pass the interview round or not.
    If there are n criteria, calculate the maximum score of those n criteria.
    If the CV has a total score of 85% of the maximum score, it passes
    Any CV with a total score below 85% does not pass
    For example, if there are 5 criteria, the maximum score will be 50, any candidate with a score greater than 85% passes.
    For CV that pass, save the CV file in the Pass folder with the name [position - candidate name] and do not send an email, for CVs that do not pass, send an email to notify the candidate who has failed.
    Only write an email to the failed candidate, remember to include the candidate's name and be polite when notifying the failure
    Finally, mark the application email as read
    """,
    agent = cv_check_biz_agent,
    context = [scoring_task],
    expected_output="Candidate CV status after calculate score"
)

hr_crew = Crew(
    agents=[email_screener_agent, cv_check_biz_agent],
    tasks=[input_task, scoring_task, selecting_task],
    verbose=True,
)

# Run the crew


In [ ]:
result = hr_crew.kickoff()
result